# Using Pandas and Metadb
The popular [Pandas](https://pandas.pydata.org/) data analysis and manipulation tool can be used with the FOLIO [MetaDB](https://librarydataplatform.org/) within a Jupyter notebook. 

## Install additional dependencies
1. Pandas `pip install pandas`
1. Install [SQLalchemy](https://www.sqlalchemy.org/) `pip install psycopg2-binary`

In [ ]:
import os
metadb_url = os.getenv("METADB_URL")
user = os.getenv("METADB_USER")
password = os.getenv("METADB_PASSWORD")

In [ ]:
import pandas as pd

from sqlalchemy import create_engine

In [ ]:
metadb_engine = create_engine("postgresql+psycopg2://{0}:{1}@{2}/{3}".format(
    user,
    password,
    metadb_url,
    "okapi"))

In [ ]:
metadb_connection = metadb_engine.connect()

## Search for Contributor Type

In [ ]:
contributor_types_sql = """SELECT id, code, name FROM folio_inventory.contributor_type__t__;"""
contributor_types_df = pd.read_sql(contributor_types_sql,
                                   con=metadb_connection)

In [ ]:
contributor_types_df

## Instances with it's Holdings Call Numbers

In [ ]:
instance_holdings_sql = """SELECT instance__t__.id, instance__t__.title, holdings_record__t__.call_number, folio_inventory.location__t__.code
FROM folio_inventory.instance__t__, folio_inventory.holdings_record__t__, folio_inventory.location__t__
WHERE folio_inventory.holdings_record__t__.instance_id = folio_inventory.instance__t__.id
AND folio_inventory.holdings_record__t__.permanent_location_id = folio_inventory.location__t__.id
LIMIT 250;"""

In [ ]:
instance_holdings_df = pd.read_sql(instance_holdings_sql,
                                   con=metadb_connection)

In [ ]:
instance_holdings_df

## Location Counts with Bar Graphs

In [ ]:
holdings_count_location_sql = """SELECT count(holdings_record__t__.id), folio_inventory.location__t__.code
FROM folio_inventory.holdings_record__t__, folio_inventory.location__t__
WHERE folio_inventory.holdings_record__t__.permanent_location_id = folio_inventory.location__t__.id
GROUP BY folio_inventory.location__t__.code;"""

In [ ]:
holdings_df = pd.read_sql(holdings_count_location_sql,
                          con=metadb_connection)

In [ ]:
holdings_100k_df = holdings_df.loc[holdings_df["count"] > 100_000]

In [ ]:
ax = holdings_100k_df.plot(kind="bar", x="code", y="count", rot=90)
ax.set_xlabel("Location")
ax.set_ylabel("Count")
ax.set_title("Holdings by Location")

In [ ]:
holdings_10 = holdings_df.loc[holdings_df["count"] <= 10]

In [ ]:
ax2 = holdings_10.plot(kind="bar", x="code", y="count", rot=90)
ax2.set_xlabel("Location")
ax2.set_ylabel("Count")
ax2.set_title("10 Holdings or Less by Location")